In [183]:
import os
from typing import *
import math
from itertools import combinations
from functools import reduce
from collections.abc import Iterable

In [2]:
def square(a: int): # a is an argument of type int of a function with a name "square"
    return a**2

square(5) # 5 is a parameter

25

In [3]:
class Foo(object):
    def __init__(
        self,
        a: int # a is an argument
    ):
        self.b = a # b is an attribute

a = Foo(1) # 1 is a parameteer

In [ ]:
[1, 2, 3]
[0, 1, 2, 3] #expand_dims(0)

[1, 0, 2, 3] #expand_dims(1)

[1, 2, 3, 0] #expand_dims(-1)

In [209]:
class Point(object):
    # takes parameters and turns it into attributes
    def __init__(
        self,
        *coordinates: float,
    ):
        self.coordinates = list(coordinates)
        self.n_dims = len(self.coordinates)

    def __len__(self) -> int:
        return self.n_dims

    def __getitem__(self, ind: int) -> float:
        return self.coordinates[ind]

    def __setitem__(self, ind: int, coord: float) -> float:
        self.coordinates[ind] = coord

    def __hash__(self):
        raise TypeError('Object "Point" is not hashable')

    def __iter__(self):
        return iter(self.coordinates)

    def __str__(self) -> str:
        return f'Point at coordinates {self.coordinates}'

    def expand_dims(self, axis: Optional[int] = 0):
        self.n_dims += 1
        if axis == -1:
            self.coordinates.insert(len(self), 0)
        else:
            self.coordinates.insert(axis, 0)


def dist(p1: Point, p2: Point) -> float:
    if len(p1) != len(p2):
        raise ValueError(f'Dimensions of the points are inconsistent: {len(p1)} != {len(p2)}.')

    return math.sqrt(sum([(coord1 - coord2)**2 for coord1, coord2 in zip(p1, p2)]))


class Segment(object):
    def __init__(
        self,
        *points: Point
    ):
        self.n_dims = max(map(
            lambda p: p.n_dims,
            points
        ))

        for point in points:
            if len(point) < self.n_dims:
                for _ in range(self.n_dims - len(point)):
                    point.expand_dims(-1)

        self.points = points

    def __len__(self) -> int:
        return len(self.points)

    def __getitem__(self, ind: int) -> float:
        return self.points[ind]

    def __iter__(self):
        return iter(self.points)

    def perimeter(self) -> float:
        return sum([dist(p1, p2) for p1, p2 in zip(self[:-1], self[1:])])

    def __str__(self) -> str:
        return f'Segment has {len(self)} points with {self.n_dims} dimentions.'


class Triangle(Segment):
    def __init__(self, p1: Point, p2: Point, p3: Point):
        super().__init__(p1, p2, p3)

    def __str__(self) -> str:
        return f'Triangle with three {self.n_dims}-dimentional points.'

    def perimeter(self) -> float:
        return sum(map(
            lambda two_points: dist(*two_points),
            combinations(self.points, 2)
        ))

    def area(self) -> float:
        p = self.perimeter() / 2
        return math.sqrt(
                p * reduce(
                lambda a, b: a*b,
                map(
                    lambda two_points: p - dist(*two_points),
                    combinations(self.points, 2)
                )
            )
        )

class Polygon(object):
    def __init__(self, s1: Segment, s2: Segment):
        self.edges = (s1, s2)
        self.points = tuple(list(s1.points) + list(s2.points))

    def __str__(self):
        return f'Polygon object'

    def __len__(self):
        return sum([len(s) for s in self.edges])

    def __iter__(self):
        return iter(self.points)

    def __contains__(self, other: Point):
        points = list(self.points) + [other]
        s1 = Segment(*points[:len(points)//2])
        s2 = Segment(*points[len(points)//2:])
        p = Polygon(s1, s2)
        return self.area() >= p.area()

    def area(self):
        return sum([
            t.area()
            for t in self.to_triangles()
        ])

    def to_triangles(self) -> list[Triangle]:
        triangles = list()
        points = self.points

        while True:
            triangle, points = self.reduce_triangle(points)
            triangles.append(triangle)

            if points is None:
                break

        return triangles

    @staticmethod
    def reduce_triangle(points: Iterable[Point]) -> tuple[Triangle, tuple[Point]] | tuple[Triangle, None]:
        if len(points) == 3:
            return Triangle(*points), None
        elif len(points) <= 3:
            raise ValueError('Splitting into triangles is impossible')
        else:
            return Triangle(*points[:3]), points[1:]


p1 = Point(1.2, 3.4, 5.6, 5.1)
p2 = Point(9.8, 8.7, 6.5, 4, 6)
p3 = Point(4.5, 8.2, 0.9)

p4 = Point(5.2, -2.4, 4.6, 5.2)
p5 = Point(2.01, -1.2, -4.1, 5.5, 2.3)
p6 = Point(7.2, 6.2, 2.6)
p7 = Point(4.9, 13.4, 6.6, 3.1)

s1 = Segment(p1, p2, p3)
s2 = Segment(p4, p5, p6, p7)
s1.perimeter()
t1 = Triangle(p1, p2, p3)
t1.perimeter()
t1.area()
po1 = Polygon(s1, s2)


In [212]:
p1 = Point(1, 0)
p2 = Point(2, 2)
p3 = Point(4, 0)

p4 = Point(4, 2)
p5 = Point(-1, 2)
p6 = Point(-1, 3)

s1 = Segment(p1, p2, p3)
s2 = Segment(p4, p5, p6)
po1 = Polygon(s1, s2)

In [68]:


arr = [4, 2, 3]

reduce(
    lambda a, b: a*b,
    arr
)

24

In [52]:

str(t1)

'Triangle with three 5-dimentional points.'

In [44]:
len(p1)

4

In [45]:
p1.n_dims

4

In [39]:
for p in s1.points:
    print(p)

Point at coordinates [1.2, 3.4, 5.6, 5.1, 0]
Point at coordinates [9.8, 8.7, 6.5, 4, 6]
Point at coordinates [4.5, 8.2, 0.9, 0, 0]


In [154]:
class User:
    def __init__(
        self,
        name: str,
        age: int,
        money: float
    ):
        self._name = name
        self._age = age
        self.__money = money

    @property
    def name(self):
        return self._name
    @name.setter
    def name(self, val):
        raise AttributeError('Can not set "name", use "rename" method')

    @property
    def age(self):
        return self._age
    @age.setter
    def age(self, val):
        raise AttributeError('Can not set "age"')

    @property
    def money(self):
        raise AttributeError('Can not get "money"')
    @money.setter
    def money(self, val):
        raise AttributeError('Can not set "money"')

    def rename(self, new_name):
        self._name = new_name

vasya = User('Vasya', 15, 100.)



In [156]:
vasya.__money

AttributeError: 'User' object has no attribute '__money'

In [141]:
vasya.name

'Vasya'

In [142]:
vasya.name = 'asasasa'

AttributeError: Can not set "name", use "rename" method

In [143]:
vasya.rename('Katya')

In [144]:
vars(vasya)

{'_name': 'Katya', 'age': 15, 'money': 100.0}

In [78]:
class Counter:
    def __init__(self) -> None:
        self._index = 0

    @property
    def index(self):
        out = self._index
        self._index += 1
        return out

    @index.setter
    def index(self, value):
        raise AttributeError('Can not set index directy')


In [88]:
c = Counter()

In [94]:
c.index, c._index

(5, 6)

In [2]:
def foo(*args, **kwargs):
    print(args)
    print(kwargs)


foo(1, 2, 3, a=4, b=5, c=6)

(1, 2, 3)
{'a': 4, 'b': 5, 'c': 6}


In [20]:
p1[1] = 2

TypeError: 'Point' object does not support item assignment

Main py errors:

KeyError - this key does not exist in the dict
ValueError - Wrong value
IndexError - this index does not exist in the list
OSError - iternal os error
FileNotFoundError
FileExistsError - this file already exists (if you are trying to create (not overwrite) existing file)
TypeError - You are trying to use this type incorrectly (f.e. trying to call "len" to the class without __len__ method)
NotImplementedError - If you are trying to use a functionality that does not exist yet
AttributeError - If you are trying to use class attribute in a wrong way
AssetionError - Error caused by "assert" operator
StopIterationError - Error to stop iteration

In [13]:
{
    p1: 'asasd'
}

TypeError: Object "Point" is not hashable